# 4.3 Machine Learning - Virtual Screening

En aquest notebook veurem com aplicar el que hem vist en l'activitat anterior per fer un **Virtual Screening**

De manera similar a com hem fet amb imatges, podem  aplicar xarxes neurals per **classificar** molècules. Només ens cal una manera de representar les molècules com un tensor, i els fingerprints que ja hem vist (MACCS o Morgan) són ideals per a fer-ho.

Ens centrarem en el **[receptor del factor de creixement epidèrmic](https://es.wikipedia.org/wiki/Receptor_del_factor_de_crecimiento_epid%C3%A9rmico)**, més conegut per les seves sigles **EGFR**. És un conegut **[oncogen](https://ca.wikipedia.org/wiki/Oncog%C3%A8n)**, que quan s'expressa en nivells alts afavoreix la conversió de la cèl·lula en tumoral. És per tant una proteïna diana de molts fàrmacs anticancerosos.

L'objectiu de l'activitat és construir un model basat en una xarxa neural que ens digui si una molècula determinada pot inhibir o no aquesta proteïna.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem

### 1. Preparació de les dades

Farem servir dades extretes de [ChEMBL](https://www.ebi.ac.uk/chembl/g/#browse/activities/filter/target_chembl_id%3ACHEMBL203) per aconseguir tots els compostos coneguts que interaccionen amb la proteïna EGFR. Amb aquestes dades entrenarem la xarxa neural. Seleccionem tots el compostos amb una activitat (pChEMBL) > 6 i els etiquetarem com a actius. Hi ha els fitxers extrets de ChEMBL al moodle.

In [3]:
egfr = pd.read_csv("egfr.chembl.tsv", sep="\t")

# Eliminem tots els compostos sense valor definit de pChEMBL i sense estructura (Smiles)
egfr = egfr[ (~pd.isnull(egfr["pChEMBL Value"])) & (~pd.isnull(egfr["Smiles"]))]

# Reanomenem i ens quedem només amb els camps que ens interessen.
columns = {
    "Molecule ChEMBL ID": "id",
    "Molecule Name": "name",
    "pChEMBL Value": "pACT",
    "Smiles": "smiles",
}

egfr = egfr.rename(columns=columns).loc[:, columns.values()]

# Eliminem si hi ha compostos duplicats, quedant-nos amb el que tingui afinitat més alta.
egfr = egfr.sort_values("pACT", ascending=False).drop_duplicates("id", keep="first")

print(f"HI HA {egfr.shape[0]:,d} compostos a ChEMBL amb afinitat per a EGFR amb estructura")

egfr.head()

HI HA 7,141 compostos a ChEMBL amb afinitat per a EGFR amb estructura


,id,name,pACT,smiles
20829,CHEMBL53711,NaN,11.22,CN(C)c1cc2c(Nc3cccc(Br)c3)ncnc2cn1
18248,CHEMBL35820,NaN,11.22,CCOc1cc2ncnc(Nc3cccc(Br)c3)c2cc1OCC
10418,CHEMBL53753,NaN,11.10,CNc1cc2c(Nc3cccc(Br)c3)ncnc2cn1
14793,CHEMBL66031,NaN,11.10,Brc1cccc(Nc2ncnc3cc4[nH]cnc4cc23)c1
24784,CHEMBL1173655,AFATINIB,11.00,CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...


In [4]:
# En seleccionem els que tinguin pChEMBL > 6. Serà el set de compostos actius.

egfr["active"] = None
egfr.loc[egfr["pACT"] > 6, "active"] = True

actives = egfr[egfr["active"] == True]

print(f"HI HA {actives.shape[0]:,d} compostos a ChEMBL actius per a EGFR")
actives.head()

HI HA 5,000 compostos a ChEMBL actius per a EGFR


,id,name,pACT,smiles,active
20829,CHEMBL53711,NaN,11.22,CN(C)c1cc2c(Nc3cccc(Br)c3)ncnc2cn1,True
18248,CHEMBL35820,NaN,11.22,CCOc1cc2ncnc(Nc3cccc(Br)c3)c2cc1OCC,True
10418,CHEMBL53753,NaN,11.10,CNc1cc2c(Nc3cccc(Br)c3)ncnc2cn1,True
14793,CHEMBL66031,NaN,11.10,Brc1cccc(Nc2ncnc3cc4[nH]cnc4cc23)c1,True
24784,CHEMBL1173655,AFATINIB,11.00,CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,True


Per a entrenar la xarxa neural també ens cal un set **negatiu**, de compostos que no tinguin afinitat per a EGFR. <br>
En aquest cas agafarem una selecció random de compostos de [ChEMBL](https://www.ebi.ac.uk/chembl/g/#browse/compounds) que no estiguin dins els que ja hem seleccionat abans. Agafarem un set de la mateixa mida que el set d'actius.

La funció `pd.sample` ens permet generar un subset de files random d'una taula.

In [5]:
chembl = pd.read_csv("chembl.tsv", sep="\t")

print(f"HI HA {chembl.shape[0]:,d} compostos a ChEMBL")

# Elimininem compostos que ja tenim a la taula anterior i compostos sense estructura.
inactives = chembl[ (~pd.isnull(chembl["Smiles"])) & (~chembl["ChEMBL ID"].isin(egfr["id"]))].sample(actives.shape[0])

columns = {
    "ChEMBL ID": "id",
    "Name": "name",
    "Smiles": "smiles",
}

inactives = inactives.rename(columns=columns).loc[:, columns.values()]
inactives = inactives.drop_duplicates("id", keep="first")
inactives["active"] = False
inactives.head()

C:\Users\300034\AppData\Local\miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


HI HA 2,157,379 compostos a ChEMBL


,id,name,smiles,active
802114,CHEMBL4778625,NaN,NS(=O)(=O)c1ccc(-n2c(SCC(=O)Nc3nc4ccc([N+](=O)...,False
1427528,CHEMBL3754828,NaN,O=C(O)C[C@H](/N=C/C=C/c1ccc(Cl)cc1)C(=O)O,False
1620732,CHEMBL3094075,NaN,Cc1ccccc1C(=O)N1C[C@@H]2CNC[C@@H](C2)C1.O=C(O)...,False
858657,CHEMBL579789,NaN,O=C(c1c(-c2[nH]c3ccccc3c2C(=O)C(F)(F)F)[nH]c2c...,False
809595,CHEMBL1389086,NaN,COc1cc(C(=O)OCC(=O)N2CC(C)OC(C)C2)cc(OC)c1OC,False


Ajuntem els sets d'actius i d'inactius i els barregem.

In [6]:
data = pd.concat((actives, inactives))
data = data.sample(frac=1)
print(data.shape)
data.head()

(10000, 5)


,id,name,pACT,smiles,active
30114,CHEMBL4441606,NaN,8.40,C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(O...,True
25938,CHEMBL4439632,NaN,8.00,C=C(Cl)C(=O)Nc1cc(Nc2nc(C)cc(-c3cn(C)c4ccccc34...,True
11765,CHEMBL1630109,NaN,7.75,O=C(/C=C/c1cccc(-c2ccc3ncnc(Nc4ccc(OCc5cccc(F)...,True
8958,CHEMBL4074601,NaN,7.92,CC(=O)N1CC[C@H](N2C(=O)N(c3ccccc3Cl)Cc3cnc(Nc4...,True
29417,CHEMBL4776012,NaN,8.49,C=C(Cl)C(=O)Nc1cccc(Nc2nc(Nc3ccc(N4CCN(C(C)=O)...,True


Com a representació de les mol·lècules farem servir els fingerprints **MACCS** generats per RDKit. <br> 
Són un array de bits **167** posicions on cada bit indica si la mol·lècula conté una subestructura determinada.

<img src=https://oi.readthedocs.io/en/latest/_images/maccs.png width=200>

In [7]:
%%time
data["fpt"] = data.apply(lambda x: np.array(Chem.MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(x["smiles"]))), axis=1)

Wall time: 21.7 s


Separarem el set total de dates (10,000 compostos, 5,000 actius i 5.000 inactius) en dos sets, el **training set** i el **test set**. <br> Agafarem un 80% per al training set in un 20% per al test set.

In [8]:
np.random.seed(0)
mask = np.random.rand(data.shape[0]) < 0.8

data["set"] = "test"
data.loc[mask, "set"] = "training" 

print(data["set"].value_counts())

data.head()

training    8028
test        1972
Name: set, dtype: int64


,id,name,pACT,smiles,active,fpt,set
30114,CHEMBL4441606,NaN,8.40,C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(O...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training
25938,CHEMBL4439632,NaN,8.00,C=C(Cl)C(=O)Nc1cc(Nc2nc(C)cc(-c3cn(C)c4ccccc34...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training
11765,CHEMBL1630109,NaN,7.75,O=C(/C=C/c1cccc(-c2ccc3ncnc(Nc4ccc(OCc5cccc(F)...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training
8958,CHEMBL4074601,NaN,7.92,CC(=O)N1CC[C@H](N2C(=O)N(c3ccccc3Cl)Cc3cnc(Nc4...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training
29417,CHEMBL4776012,NaN,8.49,C=C(Cl)C(=O)Nc1cccc(Nc2nc(Nc3ccc(N4CCN(C(C)=O)...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training


Ja nomès ens queda separar l'entrada i la sortida del model per a cada un dels sets, construir el model i entrenar-lo. <br> 
En aquest exemple farem servir la mateixa arquitectura que en l'exemple de la classificació de xifres, però la sortida en aquest cas serà binària: 0(inactiu) o 1 (actiu)

In [9]:
x_train = np.array([np.array(fpt) for fpt in data.loc[data["set"] == "training", "fpt"]])
y_train = np.array(data.loc[data["set"] == "training", "active"].values, dtype=int)

x_test = np.array([np.array(fpt) for fpt in data.loc[data["set"] == "test", "fpt"]])
y_test = np.array(data.loc[data["set"] == "test", "active"].values, dtype=int)

print(f"Dimensions del tensor d'entrada (train): {x_train.shape}")
print(f"Dimensions del tensor de sortida (train): {y_train.shape}")
print()
print(f"Dimensions del tensor d'entrada (test): {x_test.shape}")
print(f"Dimensions del tensor de sortida (test): {y_test.shape}")

Dimensions del tensor d'entrada (train): (8028, 167)
Dimensions del tensor de sortida (train): (8028,)

Dimensions del tensor d'entrada (test): (1972, 167)
Dimensions del tensor de sortida (test): (1972,)


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

C:\Users\300034\AppData\Local\miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\300034\AppData\Local\miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\300034\AppData\Local\miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\3

In [11]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(167,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 167)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               21504     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 21,633
Trainable params: 21,633
Non-trainable params: 0
_________________________________________________________________


In [12]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
8028/8028 [==============================] - 0s 49us/sample - loss: 0.5954 - acc: 0.8247
Epoch 2/5
8028/8028 [==============================] - 0s 29us/sample - loss: 0.5601 - acc: 0.8861
Epoch 3/5
8028/8028 [==============================] - 0s 29us/sample - loss: 0.5551 - acc: 0.8921
Epoch 4/5
8028/8028 [==============================] - 0s 32us/sample - loss: 0.5522 - acc: 0.8969
Epoch 5/5
8028/8028 [==============================] - 0s 29us/sample - loss: 0.5500 - acc: 0.9010


In [14]:
model.evaluate(x_test,  y_test, verbose=2)

1972/1972 - 0s - loss: 0.5523 - acc: 0.8808


[0.5522734209683555, 0.88083166]

Sembla que el model s'ha entrenat correctament, amb una exactitud al voltant del 90% al set de test.

Per veure com s'aplica el model per fer noves prediccions, l'aplicarem primer a una sola mol·ècula, i després totes les dades que tenim. Veiem que el model ens retorna un sol valor de 0 a 1. Tallem a 0.5 per separar prediccions actives de inactives.


In [15]:
x = np.array([x for x in data["fpt"][0]]).reshape(1, -1)
model.predict(x)

array([[0.9989229]], dtype=float32)

In [16]:
x = np.array([x for  x in data["fpt"]])
data["predicted"] = model.predict(x) > 0.5

data.head()

,id,name,pACT,smiles,active,fpt,set,predicted
30114,CHEMBL4441606,NaN,8.40,C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(O...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training,True
25938,CHEMBL4439632,NaN,8.00,C=C(Cl)C(=O)Nc1cc(Nc2nc(C)cc(-c3cn(C)c4ccccc34...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training,True
11765,CHEMBL1630109,NaN,7.75,O=C(/C=C/c1cccc(-c2ccc3ncnc(Nc4ccc(OCc5cccc(F)...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training,True
8958,CHEMBL4074601,NaN,7.92,CC(=O)N1CC[C@H](N2C(=O)N(c3ccccc3Cl)Cc3cnc(Nc4...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training,True
29417,CHEMBL4776012,NaN,8.49,C=C(Cl)C(=O)Nc1cccc(Nc2nc(Nc3ccc(N4CCN(C(C)=O)...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",training,True


<mark> **Pregunta 7**: Agafem un subset random del test set de 100 compostos. Quina és la **precisió**, **recall**, i **F1-measure** del model en aquests 100 compostos ? </mark>

Precisió = True Positives / (True Positives + False Positives)

Recall = True Positives / (True Positives + False Negatives)

F1-measure = 2 * (Precisió * Recall) / (Precisió + Recall)

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [19]:
subset = data[data["set"] == "test"].sample(100)
subset.head()

,id,name,pACT,smiles,active,fpt,set,predicted
23279,CHEMBL3979366,NaN,8.18,C=CC(=O)N1CC[C@H](Oc2cc3c(Nc4ccc(F)c(Cl)c4F)nc...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,True
1612216,CHEMBL2347560,NaN,NaN,O=S(=O)(c1ccc(-c2ccc3ncnc(Nc4ccc(OCc5cccc(Br)c...,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,True
1656,CHEMBL3815161,NaN,8.54,O=C(NCCN1CCOCC1)Nc1ccc2ncnc(Nc3ccc(F)c(Cl)c3)c2c1,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,False
4068,CHEMBL3671542,NaN,6.99,C=CC(=O)Nc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCCNC(=...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,True
10202,CHEMBL3813728,NaN,8.65,Fc1cccc(COc2ccc(Nc3ncnc4ccc(NC(=S)NCCN5CCOCC5)...,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,True


In [26]:
true_labels = subset["active"]
precision = subset["predicted"]

true_labels = true_labels.astype(int)
precision = precision.astype(int)

# Càlcul de les mètriques
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print("Precisió:", precision)
print("Recall:", recall)
print("F1-measure:", f1)

Precisió: 0.9534883720930233
Recall: 0.7592592592592593
F1-measure: 0.845360824742268
